<a href="https://colab.research.google.com/github/Abhishekconfused/Models/blob/main/Wordembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
import tensorflow

In [3]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
from tensorflow.keras.models import Sequential#type
from tensorflow.keras.layers import Dense#final layer
from tensorflow.keras.layers import Flatten#converts into one
from tensorflow.keras.layers import Embedding#more hidden layer

In [5]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

In [6]:
sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

In [7]:
one_hot('amazing restaurant',30)

[3, 5]

In [8]:
max_vocab=30
encoded_reviews = [one_hot(d,max_vocab) for d in reviews]
print(encoded_reviews)

[[27, 19], [3, 5], [21, 18], [9, 1, 5], [3, 10, 28], [26, 19], [23, 10, 27], [10, 20], [10, 1], [6, 25]]


In [9]:
max_length=3
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)

[[27 19  0]
 [ 3  5  0]
 [21 18  0]
 [ 9  1  5]
 [ 3 10 28]
 [26 19  0]
 [23 10 27]
 [10 20  0]
 [10  1  0]
 [ 6 25  0]]


In [10]:
embedded_vector_size=5
model = Sequential()
model.add(Embedding(max_vocab, embedded_vector_size, input_length=max_length,name="embedding"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [11]:
x=padded_reviews
y=sentiment

In [12]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 5)              150       
                                                                 
 flatten (Flatten)           (None, 15)                0         
                                                                 
 dense (Dense)               (None, 1)                 16        
                                                                 
Total params: 166 (664.00 Byte)
Trainable params: 166 (664.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.fit(x,y,epochs=50,verbose=0)

In [15]:
model.evaluate(x,y)

1/1 [==============================] - 0s 314ms/step - loss: 0.6238 - accuracy: 1.0000


[0.6238259673118591, 1.0]

In [16]:
weights = model.get_layer('embedding').get_weights()[0]
len(weights)

30

In [17]:
weights[3]

array([ 0.03095971, -0.07862575,  0.07021564,  0.00674393,  0.06542142],
      dtype=float32)

In [18]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.0 MB/s eta 0:00:00


In [19]:
import gensim
import pandas as pd

In [22]:
df=pd.read_csv('/content/drive/MyDrive/train (1).csv')

In [23]:
df.head()

,id,content,type
0,0,informed equifax numerous time account amount ...,credit_reporting
1,1,writing regard late payment credit report bank...,debt_collection
2,2,incorrectly report balance report asked experi...,credit_reporting
3,3,attempted investigate remove inaccurate inform...,credit_reporting
4,4,trying trade vehicle newer model told salesman...,credit_reporting


In [25]:
df["content"][0]

'informed equifax numerous time account amount result identity theft yet fail block information account im confused reason equifax fails follow law fcra b victim identity theft account credit file result act request block account accordance fcra certifies statement information regarding amount information account relating transaction created'

In [26]:
gensim.utils.simple_preprocess(df["content"][0])

['informed',
 'equifax',
 'numerous',
 'time',
 'account',
 'amount',
 'result',
 'identity',
 'theft',
 'yet',
 'fail',
 'block',
 'information',
 'account',
 'im',
 'confused',
 'reason',
 'equifax',
 'fails',
 'follow',
 'law',
 'fcra',
 'victim',
 'identity',
 'theft',
 'account',
 'credit',
 'file',
 'result',
 'act',
 'request',
 'block',
 'account',
 'accordance',
 'fcra',
 'certifies',
 'statement',
 'information',
 'regarding',
 'amount',
 'information',
 'account',
 'relating',
 'transaction',
 'created']

In [28]:
df.isnull().sum()

id         0
content    1
type       0
dtype: int64

In [29]:
df.dropna(inplace=True)

In [30]:
text=df.content.apply(gensim.utils.simple_preprocess)
text

0        [informed, equifax, numerous, time, account, a...
1        [writing, regard, late, payment, credit, repor...
2        [incorrectly, report, balance, report, asked, ...
3        [attempted, investigate, remove, inaccurate, i...
4        [trying, trade, vehicle, newer, model, told, s...
                               ...                        
32480    [sent, correct, information, credit, agency, u...
32481    [scheduled, appointment, keybank, discus, unem...
32482    [year, wondering, credit, never, consistent, n...
32483    [last, day, working, tomorrow, full, week, wor...
32484    [looked, score, three, major, credit, bureau, ...
Name: content, Length: 32484, dtype: object

In [31]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [33]:
model.build_vocab(text, progress_per=10)

In [34]:
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11438058, 12924950)

In [35]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [36]:
model.wv.most_similar("bad")

[('ruined', 0.6530683636665344),
 ('decent', 0.6387496590614319),
 ('poor', 0.6007210612297058),
 ('badly', 0.5955667495727539),
 ('horrible', 0.592562198638916),
 ('destroying', 0.5921496748924255),
 ('ruining', 0.5911309719085693),
 ('ruin', 0.5789188146591187),
 ('truly', 0.5775256752967834),
 ('terrible', 0.5774336457252502)]

In [38]:
model.wv.similarity(w1="bank", w2="card")

0.26958182

In [39]:
model.wv.similarity(w1="great", w2="good")

0.27598754